In [1]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt

import time #to have today's date
timestr = time.strftime("%Y%m%d")

In [2]:
#get the LDA model from another notebook
%run ./p_07_final_LDA_paper.ipynb

#doc_term_matrix_x 
#ldamodel 
#doc_list_paras_x

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\mann\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\mann\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


## find the dominant topic for each text
https://www.machinelearningplus.com/nlp/topic-modeling-gensim-python/#13viewthetopicsinldamodel

In [17]:
def format_topics_sentences(ldamodel=ldamodel, doc_term_matrix=doc_term_matrix_x, texts=doc_list_paras_x): 
    # Init output
    sent_topics_df = pd.DataFrame()

    # Get main topic in each document
    #i are all the document numbers, row has all the ocurring topics and their percentage share
    for i, row in enumerate(ldamodel[doc_term_matrix]):
        row = sorted(row, key=lambda x: (x[1]), reverse=True) #sort the topics of each document by perc contribution
        # Get the Dominant topic, Perc Contribution and Keywords for each document
        for j, (topic_num, prop_topic) in enumerate(row):
            if j == 0:  # => dominant topic
                wp = ldamodel.show_topic(topic_num, topn=20) #topn is number of words
                topic_keywords = ", ".join([word for word, prop in wp])
                sent_topics_df = sent_topics_df.append(pd.Series([int(topic_num), round(prop_topic,4),\
                                                                  topic_keywords]), ignore_index=True)
            else:
                break
    sent_topics_df.columns = ['Dominant_Topic', 'Perc_Contribution', 'Topic_Keywords']

    # Add original text to the end of the output
    contents = pd.Series(texts)
    sent_topics_df = pd.concat([sent_topics_df, contents], axis=1)
    return(sent_topics_df)

In [18]:
df_topic_sents_keywords = format_topics_sentences(ldamodel=ldamodel,\
                                                  doc_term_matrix=doc_term_matrix_x, texts=doc_list_paras_x)

# Format
df_dominant_topic = df_topic_sents_keywords.reset_index()
df_dominant_topic.columns = ['Document_No', 'Dominant_ori_Topic', 'Topic_Perc_Contrib', 'keywords', 'Text']

MemoryError: Unable to allocate 1.36 MiB for an array with shape (22, 16163) and data type float32

In [ ]:
df_dominant_topic.head(5)

### get all the topic scores for each document

In [ ]:
#in the code above we just get the dominant topic for each document.
#For the manual topic categorization I want the top 10 documents for each topic, not only if they are dominant!

In [ ]:
def get_df_topics_per_doc(sent_lda, sent_doc_term_matrix, texts):
    df_x = pd.DataFrame()
    # Get relevant topics for each document, this takes a while
    for i, row in enumerate(sent_lda[sent_doc_term_matrix]): #this provides a list with all topics and their percentage contribution
        # Get all the topics, Perc Contribution and Keywords for each document
        for j in row:
            topic_num = j[0]
            prop_topic = j[1]
            df_x.loc[i, 't'+str(topic_num)] = prop_topic
    
    # Add original text to the end of the output
    contents = pd.Series(texts)
    df_x_texts = pd.concat([df_x, contents], axis=1).rename(columns={0:'Text'})
    return(df_x_texts)

In [ ]:
df_topics_per_doc = get_df_topics_per_doc(ldamodel, doc_term_matrix_x, doc_list_paras_x)

In [ ]:
df_top_docs_per_topic = pd.DataFrame()
for t in list(range(0,var_no_topics)):
    df_t = pd.DataFrame()
    df_t = df_topics_per_doc.nlargest(20, 't'+str(t))[['t'+str(t), 'Text']].\
            rename(columns={'t'+str(t):'Topic_Perc_Contrib'})
    df_t['ori_topic_num'] = t
    #check: each topic has 20 documents?
    var_len = len(df_t) 
    if var_len != 20:
        print('topic ' + t + 'has only ' + var_len + ' documents')
    df_top_docs_per_topic = df_top_docs_per_topic.append(df_t)
    df_top_docs_per_topic.reset_index(drop=True, inplace=True)

In [ ]:
#get keywords for each topic!
df_topics_keywords = pd.DataFrame({'ori_topic_num':list(range(0,var_no_topics)), 'keywords': list_topics_keywords})
df_top_docs_per_topic_keyw = df_top_docs_per_topic.merge(df_topics_keywords)

In [ ]:
df_top_docs_per_topic_keyw.to_csv('.//results//topics_group//qa_topic_keywords_representative_text_'+timestr+'_V01.csv',
    sep=';', decimal=',')

## find the most representative document for each topic

In [ ]:
# Group top 5 sentences under each topic
sent_topics_sorteddf_mallet = pd.DataFrame()

sent_topics_outdf_grpd = df_topic_sents_keywords.groupby('Dominant_Topic')

for i, grp in sent_topics_outdf_grpd: #i are the topic numbers. grp 
    sent_topics_sorteddf_mallet = pd.concat([sent_topics_sorteddf_mallet, 
        grp.sort_values(['Perc_Contribution'], ascending=[0]).head(10)], axis=0) #10 most representative documents

# Reset Index    
sent_topics_sorteddf_mallet.reset_index(drop=True, inplace=True)

# Format
sent_topics_sorteddf_mallet.columns = ['ori_topic_num', "Topic_Perc_Contrib", "keywords", "Text"]

In [ ]:
sent_topics_sorteddf_mallet.to_csv('.//results//topics_group//qa_topic_dominant_keywords_representative_text_'+timestr+'_V01.csv',
    sep=';', decimal=',')